# Hydropower Simulations with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways.
This example demonstrates a few of the options for modeling hydropower generation.

## Dependencies

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))

"/Users/cbarrows/Documents/repos/SIIPExamples.jl"

### Modeling Packages

In [2]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using JuMP
using Cbc # solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Data
There is a meaningless test dataset assembled in the
[make_hydropower_data.jl](../../script/PowerSimulations_examples/make_hydro_data.jl) script.

In [5]:
include(joinpath(pkgpath, "script", "PowerSimulations_examples", "make_hydro_data.jl"))

┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = voltage
│   field_value = 1.07
│   valid_range = voltagelimits
│   valid_info.ist_struct = PowerSystems.Bus(6, "Bus 6", PowerSystems.BusTypes.PV, -0.24818581963359368, 1.07, (min = 0.94, max = 1.06), 13.8, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("7f60e2a5-59d2-4607-9207-7c3ba70b72dd"), nothing))
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/yQ40g/src/validation.jl:209
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = voltage
│   field_value = 1.062
│   valid_range = voltagelimits
│   valid_info.ist_struct = PowerSystems.Bus(7, "Bus 7", PowerSystems.BusTypes.PQ, -0.23335052099164186, 1.062, (min = 0.94, max = 1.06), 13.8, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("764b1fb8-f5ea-4180-b897-0aeca54b2681"), nothing))
└ @ InfrastructureSystems /Users/cba

## Two PowerSimulations features determine hydropower representation.
There are two principal ways that we can customize hydropower representation in
PowerSimulations. First, we can play with the formulation applied to hydropower generators
using the `DeviceModel`. We can also adjust how simulations are configured to represent
different decision making processes and the information flow between those processes.

### Hydropower `DeviceModel`s

First, the assignment of device formulations to particular device types gives us control
over the representation of devices. This is accomplished by defining `DeviceModel`
instances. For hydro power representations, we have two available generator types in
PowerSystems:

In [6]:
TypeTree(PSY.HydroGen)

PowerSystems.HydroGen
├──PowerSystems.HydroDispatch
└──PowerSystems.HydroEnergyReservoir

And in PowerSimulations, we have several available formulations that can be applied to
the hydropower generation devices:

In [7]:
TypeTree(PSI.AbstractHydroFormulation, scopesep="\n", init_expand = 5)

PowerSimulations
AbstractHydroFormulation
├──PowerSimulations
│  AbstractHydroDispatchFormulation
│  ├──PowerSimulations
│  │  HydroDispatchRunOfRiver
│  └──PowerSimulations
│     AbstractHydroReservoirFormulation
│     ├──PowerSimulations
│     │  HydroDispatchReservoirStorage (0 children)
│     └──PowerSimulations
│        HydroDispatchReservoirFlow (0 children)
├──PowerSimulations
│  HydroFixed
└──PowerSimulations
   AbstractHydroUnitCommitment

Let's see what some of the different combinations create. First, let's apply the
`HydroDispatchRunOfRiver` formulation to the `HydroEnergyReservoir` generators, and the
`HydroFixed` formulation to `HydroDispatch` generators.
 - The `HydroFixed` formulation just acts
like a load subtractor, forcing the system to accept it's generation.
 - The `HydroDispatchRunOfRiver` formulation represents the the energy flowing out of
a reservoir. The model can choose to produce power with that energy or just let it spill by.

In [8]:
devices = Dict{Symbol,DeviceModel}(
    :Hyd1 => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :Hyd2 => DeviceModel(HydroDispatch, HydroFixed),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      Hyd1:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Hyd2:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroFixed
  branches: 
  services: 


Now we can see the resulting JuMP model:

In [9]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

The first two constraints are the power balance constraints that require the generation
from the controllable `HydroEnergyReservoir` generators to be equal to the load (flat 10.0 for all time periods)
minus the generation from the `HydroDispatch` generators [1.97, 1.983, ...]. The 3rd and 4th
constraints limit the output of the `HydroEnergyReservoir` generator to the limit defined by the
`max_activepwoer` forecast. And the last 4 constraints are the lower and upper bounds of
the `HydroEnergyReservoir` operating range.

Next, let's apply the `HydroDispatchReservoirFlow` formulation to the `HydroEnergyReservoir` generators.

In [10]:
devices = Dict{Symbol,DeviceModel}(
    :Hyd1 => DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirFlow),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      Hyd1:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirFlow
  branches: 
  services: 


And, the resulting JuMP model:

In [11]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Finally, let's apply the `HydroDispatchReservoirStorage` formulation to the `HydroEnergyReservoir` generators.

In [12]:
devices = Dict{Symbol,DeviceModel}(
    :Hyd1 => DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirStorage),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      Hyd1:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirStorage
  branches: 
  services: 


In [13]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

### Multi-Stage `SimulationSequence`
The purpose of a multi-stage simulation is to represent scheduling decisions consistently
with the time scales that govern different elements of power systems.

Multi-Day to Daily Simulation:

In the multi-day model, we'll use a really simple representation of all system devices
so that we can maintain computational tractability while getting an estimate of system
requirements/capabilities.

In [14]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatchNoMin),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroEnergyReservoir => DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirStorage),
)
template_md = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

For the daily model, we can increase the modeling detail since we'll be solving shorter
problems.

In [15]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatchNoMin),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroEnergyReservoir => DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirStorage),
)
template_da = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

In [16]:
stages_definition = Dict(
    "MD" => Stage(GenericOpProblem, template_md, c_sys5_hy_wk, solver),
    "DA" => Stage(GenericOpProblem, template_da, c_sys5_hy_uc, solver),
)

Dict{String,PowerSimulations.Stage{PowerSimulations.GenericOpProblem}} with 2 entries:
  "DA" => Stage()…
  "MD" => Stage()…

This builds the sequence and passes the the energy dispatch schedule for the `HydroEnergyReservoir`
generator from the "MD" stage to the "DA" stage in the form of an energy limit over the
synchronized periods.

In [17]:
sequence = SimulationSequence(
    step_resolution = Hour(48),
    order = Dict(1 => "MD", 2 => "DA"),
    feedforward_chronologies = Dict(("MD" => "DA") => Synchronize(periods = 2)),
    horizons = Dict("MD" => 2, "DA" => 24),
    intervals = Dict(
        "MD" => (Hour(48), Consecutive()),
        "DA" => (Hour(24), Consecutive()),
    ),
    feedforward = Dict(
        ("DA", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_from_stage = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
    ),
    cache = Dict( ("MD", "DA") => StoredEnergy(PSY.HydroEnergyReservoir, PSI.ENERGY)),
    ini_cond_chronology = InterStageChronology(),
);

In [18]:
file_path = tempdir()

sim = Simulation(
    name = "hydro",
    steps = 1,
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
)

build!(sim)

┌ Info: Building Stage 2-DA
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/32SZy/src/core/simulation.jl:354
┌ Info: Building Stage 1-MD
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/32SZy/src/core/simulation.jl:354
┌ Info: 
│  ───────────────────────────────────────────────────────────────────────────
│                                     Time                   Allocations      
│                             ──────────────────────   ───────────────────────
│       Tot / % measured:          619ms / 100%            50.7MiB / 100%     
│ 
│  Section            ncalls     time   %tot     avg     alloc   %tot      avg
│  ───────────────────────────────────────────────────────────────────────────
│  Build Simulation        1    619ms   100%   619ms   50.7MiB  100%   50.7MiB
│    Build Stage DA        1    488ms  78.7%   488ms   39.8MiB  78.5%  39.8MiB
│    Build Stage MD        1   6.27ms  1.01%  6.27ms    292KiB  0.56%   292KiB
│  ────────────

We can look at the "MD" Model

In [19]:
sim.stages["MD"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 16
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 12 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 16 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 14 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [20]:
sim.stages["DA"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 192
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 144 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 145 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 192 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 168 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can execute the simulation by running the following command
```julia

In [21]:
#sim_results = execute!(sim)
#```

3-Stage Simulation:

In [22]:
stages_definition = Dict(
    "MD" => Stage(GenericOpProblem, template_md, c_sys5_hy_wk, solver),
    "DA" => Stage(GenericOpProblem, template_da, c_sys5_hy_uc, solver),
    "ED" => Stage(GenericOpProblem, template_da, c_sys5_hy_ed, solver),
)

sequence = SimulationSequence(
    step_resolution = Hour(48),
    order = Dict(1 => "MD", 2 => "DA", 3 => "ED"),
    feedforward_chronologies = Dict(
        ("MD" => "DA") => Synchronize(periods = 2),
        ("DA" => "ED") => Synchronize(periods = 24),
    ),
    intervals = Dict(
        "MD" => (Hour(48), Consecutive()),
        "DA" => (Hour(24), Consecutive()),
        "ED" => (Hour(1), Consecutive()),
    ),
    horizons = Dict("MD" => 2, "DA" => 24, "ED" => 12),
    feedforward = Dict(
        ("DA", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_from_stage = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
        ("ED", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_from_stage = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
    ),
    cache = Dict( ("MD", "DA") => StoredEnergy(PSY.HydroEnergyReservoir, PSI.ENERGY)),
    ini_cond_chronology = InterStageChronology(),
);

In [23]:
sim = Simulation(
    name = "hydro",
    steps = 1,
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
)

Simulation()


In [24]:
build!(sim)

┌ Info: Building Stage 2-DA
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/32SZy/src/core/simulation.jl:354
┌ Info: Building Stage 3-ED
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/32SZy/src/core/simulation.jl:354
┌ Info: Building Stage 1-MD
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/32SZy/src/core/simulation.jl:354
┌ Info: 
│  ───────────────────────────────────────────────────────────────────────────
│                                     Time                   Allocations      
│                             ──────────────────────   ───────────────────────
│       Tot / % measured:         21.0ms / 100%            2.91MiB / 100%     
│ 
│  Section            ncalls     time   %tot     avg     alloc   %tot      avg
│  ───────────────────────────────────────────────────────────────────────────
│  Build Simulation        1   21.0ms   100%  21.0ms   2.91MiB  100%   2.91MiB
│    Build Stage ED        1   7.14ms  34.

We can look at the "MD" Model

In [25]:
sim.stages["MD"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 16
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 12 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 16 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 14 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [26]:
sim.stages["DA"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 192
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 144 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 145 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 192 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 168 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "ED" model

In [27]:
sim.stages["ED"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 96
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 24 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 72 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 73 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 96 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 84 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*